In [60]:
import pandas as pd
pd.set_option('display.max_colwidth', None)


In [61]:
airr = pd.read_table('/home/michaelswift/TabulaSapiens/data/snkmk_outs/combined_igblast.airr.tsv')

/home/michaelswift/miniconda3/envs/scirpy/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (93,94) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [62]:
airr.productive == 'T'

# get rid of merging artefacts
print(airr.shape[0])
airr = airr.dropna(subset=['locus'])
print(airr.shape[0], "removed presumed merging artefacts")
airr = airr[airr.productive == "T"]
print(airr.shape[0], "removed unproductive chains")

print("{} assemblies from TS".format(airr.shape[0]))

#df = pd.read_csv('notebooks/Clonality/BCRChangeo.csv', index_col=0)
df = airr
#10X
df.loc[:,"cell_id"] = df.sequence_id.replace('_contig_[0-9]', '', regex=True)
df.cell_id.replace("\-","_", regex=True, inplace=True)
# 10X
df.cell_id.replace("_5prime..CR", '_5prime', regex = True, inplace = True)
df.cell_id.replace('_[0-9]_[T][S]', '_TS',regex = True, inplace = True)
df.cell_id.replace('10X_[0-9]_[0-9]', '10X',regex = True, inplace = True)
df.loc[:, 'cell_id'] = df.cell_id.str.rsplit(" ", expand=True)[0]

df.head().cell_id.values[-20:]
# Munge the data a little
df.loc[:, 'cell_id'] = df.cell_id.str.split('|', expand=True)[0]

21079
21079 removed presumed merging artefacts
19269 removed unproductive chains
19269 assemblies from TS


## TSP metadata

In [63]:
met = pd.read_csv('../data/TSP1_TSP15_metadata.csv')
#met = pd.read_csv('Immune_Obs.csv')

# yet another corner case currently not worth fixing this, maybe angela can do it
met[(met.donor == "TSP1") & (met.method == 'smartseq2')].cell_id.str.rsplit(".", n = 5, expand = True)[0]


met['_cell_id'] = met['cell_id']
#met['_cell_id'] = met['_cell_id'].str.replace('_[SL][0-9][0-9][]', '', regex = True)
met['_cell_id'] = met['_cell_id'].str.replace('_[0-2]_[0-2]_5prime', '_5prime', regex = True)
#met['_cell_id'].replace('_L[0-9]..', '',regex = True, inplace = True)
#met['_cell_id'].replace('_S[0-9]..', '',regex = True, inplace = True)

/home/michaelswift/miniconda3/envs/scirpy/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (10,14,15,16,17,18) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [64]:
merged = pd.merge(df, met, how = 'inner', left_on='cell_id', right_on='_cell_id')

In [65]:
print(merged.shape[0], "assmeblies remain after merging")

10446 assmeblies remain after merging


In [66]:
met.groupby('donor').tissue.value_counts()

donor  tissue         
TSP1   Lung               13211
       Bladder            11736
       Pancreas            6798
       Blood               4020
       Muscle              3760
TSP10  Fat                12768
       Skin                5875
       Blood               5028
TSP11  Bone_Marrow         2514
TSP12  Heart              11505
TSP13  Bone_Marrow          543
TSP14  Lymph_Node         30427
       Thymus             21578
       Spleen             18965
       Salivary_Gland     17858
       Lung               11971
       Large_Intestine    11439
       Small_Intestine    10458
       Blood               9964
       Vasculature         8684
       Muscle              8404
       Fat                 7696
       Bone_Marrow         6095
       Tongue              5428
       Liver               4496
       Skin                3563
       Prostate            3282
       Bladder             3066
TSP15  Eye                 5651
TSP2   Lung               18674
       Muscle    

In [67]:
met[(met['tissue'] == 'Blood') & (met['method'] =='smartseq2')].groupby('donor').cell_ontology_class.value_counts()

donor  cell_ontology_class                                        
TSP1   cd4-positive, alpha-beta t cell                                158
       naive b cell                                                    35
       cd8-positive, alpha-beta t cell                                 26
       memory b cell                                                   25
       monocyte                                                        18
       nk cell                                                         11
       erythrocyte doublet                                              5
       classical monocyte                                               4
       cd4-positive helper t cell                                       3
       intermediate monocyte                                            2
       cd24 neutrophil                                                  1
       erythrocyte                                                      1
TSP2   monocyte                              

In [68]:
airr[airr.library == 'tracer'].cell_id.tail()

21072     TSP8_Prostate_NA_SS2_B134140_B134704_Epithelial_B14_L002
21074     TSP8_Prostate_NA_SS2_B134140_B134704_Epithelial_B14_L002
21075    TSP8_Prostate_NA_SS2_B134137_B134703_Endothelial_B18_L002
21076      TSP8_Prostate_NA_SS2_B134140_B134704_Epithelial_D7_L002
21078         TSP8_Prostate_NA_SS2_B134141_B134697_Immune_H12_L001
Name: cell_id, dtype: object

In [69]:
airr.locus.value_counts()

TRB    6733
TRA    6096
IGH    2574
IGL    2155
IGK    1711
Name: locus, dtype: int64

In [71]:
merged[merged.tissue == 'Blood'].locus.value_counts()

TRB    628
TRA    547
Name: locus, dtype: int64

In [72]:
pd.DataFrame(merged.groupby('tissue')['locus'].value_counts())

locus
tissue      locus       
Bladder     TRA       16
            TRB       14
            IGH        1
            IGK        1
Blood       TRB      628
...                  ...
Trachea     TRB        1
Uterus      TRA       54
            TRB       50
Vasculature TRA       24
            TRB       20

[75 rows x 1 columns]

In [73]:
merged.to_csv('../data/merged_airr.tsv', sep = '\t', index=False)

In [75]:
airr.cell_id.str.split('_', expand=True)[2].value_counts()

BM                    5929
NA                    4273
Thymus                2147
supradiaphragmatic    1310
vertebralbody         1053
                      ... 
S283                     1
S258                     1
S10                      1
S208                     1
S166                     1
Name: 2, Length: 203, dtype: int64

In [76]:
merged[merged.cell_ontology_class.str.contains('b cell|plasma cell')].groupby('donor').tissue.value_counts()

donor  tissue        
TSP10  Fat                13
       Skin               10
TSP11  Bone_Marrow       314
TSP13  Bone_Marrow       155
TSP2   Spleen            355
       Bone_Marrow       271
       Kidney             60
       Lymph_Node         48
       Bladder             8
       Lung                8
       Skin                4
       Blood               2
TSP3   Eye                18
TSP4   Mammary             2
TSP5   Eye                25
TSP6   Trachea            35
       Liver               3
TSP7   Salivary_Gland      6
       Lymph_Node          5
       Blood               1
       Spleen              1
Name: tissue, dtype: int64

In [202]:
merged

,sequence_id,sequence,locus,stop_codon,vj_in_frame,productive,rev_comp,v_call,d_call,j_call,...,10X_barcode,cDNAplate,libraryplate,well,notes,seqrun,cell_identifier,n_counts,n_genes,_cell_id
0,AAAGTAGGTGTGGCTC-1_contig_1_TSP2_BM_vertebralb...,TGGGGGTAGCTCAGGAAGCAGAGCCTGGAGCATCTCCACTATGGCC...,IGL,F,T,T,F,IGLV6-57*01,NaN,IGLJ3*02,...,AAAGTAGGTGTGGCTC_TSP2_BM_vertebralbody_10X_2_2...,NaN,NaN,NaN,5prime,2,AAAGTAGGTGTGGCTC_TSP2_BM_vertebralbody_10X_2_2...,15471.0,1484,AAAGTAGGTGTGGCTC_TSP2_BM_vertebralbody_10X_5prime
1,AAAGTAGGTGTGGCTC-1_contig_1_TSP2_BM_vertebralb...,GGGGGGTAGCTCAGGAAGCAGAGCCTGGAGCATCTCCACTATGGCC...,IGL,F,T,T,F,IGLV6-57*01,NaN,IGLJ3*02,...,AAAGTAGGTGTGGCTC_TSP2_BM_vertebralbody_10X_2_2...,NaN,NaN,NaN,5prime,2,AAAGTAGGTGTGGCTC_TSP2_BM_vertebralbody_10X_2_2...,15471.0,1484,AAAGTAGGTGTGGCTC_TSP2_BM_vertebralbody_10X_5prime
2,AACTCAGCAGCCAATT-1_contig_1_TSP2_BM_vertebralb...,TGGGGGCGCCAGGGGTCCCTTCCAATATCAGCACCATGGCCTGGAC...,IGL,F,T,T,F,IGLV7-43*01,NaN,IGLJ3*02,...,AACTCAGCAGCCAATT_TSP2_BM_vertebralbody_10X_1_2...,NaN,NaN,NaN,5prime,2,AACTCAGCAGCCAATT_TSP2_BM_vertebralbody_10X_1_2...,16216.0,3780,AACTCAGCAGCCAATT_TSP2_BM_vertebralbody_10X_5prime
3,AACTCAGCAGCCAATT-1_contig_2_TSP2_BM_vertebralb...,TGGGGATCATCCAACAACCACATCCCTTCTCTACAGAAGCCTCTGA...,IGH,F,T,T,F,IGHV1-46*01,"IGHD2-15*01,IGHD2-21*01,IGHD2-21*02",IGHJ4*02,...,AACTCAGCAGCCAATT_TSP2_BM_vertebralbody_10X_1_2...,NaN,NaN,NaN,5prime,2,AACTCAGCAGCCAATT_TSP2_BM_vertebralbody_10X_1_2...,16216.0,3780,AACTCAGCAGCCAATT_TSP2_BM_vertebralbody_10X_5prime
4,AACTCAGCAGCCAATT-1_contig_1_TSP2_BM_vertebralb...,AGATGGGGGATGATAGAAGTAACTCTTATAACTGGAGGTTGCAGGT...,TRA,F,T,T,F,TRAV13-1*02,NaN,TRAJ11*01,...,AACTCAGCAGCCAATT_TSP2_BM_vertebralbody_10X_1_2...,NaN,NaN,NaN,5prime,2,AACTCAGCAGCCAATT_TSP2_BM_vertebralbody_10X_1_2...,16216.0,3780,AACTCAGCAGCCAATT_TSP2_BM_vertebralbody_10X_5prime
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10441,TSP8_Prostate_NA_SS2_B134140_B134704_Epithelia...,CAGAAGGAGGTGGAGCAGAATTCTGGACCCCTCAGTGTTCCAGAGG...,TRA,F,T,T,F,TRAV12-2*01,NaN,TRAJ47*01,...,NaN,B134140,B134704,B14,Epithelial,NaN,TSP8_Prostate_NA_SS2_B134140_B134704_Epithelia...,1602157.0,1937,TSP8_Prostate_NA_SS2_B134140_B134704_Epithelia...
10442,TSP8_Prostate_NA_SS2_B134140_B134704_Epithelia...,GACACAGCTGTTTCCCAGACTCCAAAATACCTGGTCACACAGATGG...,TRB,F,T,T,F,TRBV3-1*01,TRBD1*01,TRBJ2-1*01,...,NaN,B134140,B134704,B14,Epithelial,NaN,TSP8_Prostate_NA_SS2_B134140_B134704_Epithelia...,1602157.0,1937,TSP8_Prostate_NA_SS2_B134140_B134704_Epithelia...
10443,TSP8_Prostate_NA_SS2_B134137_B134703_Endotheli...,GACACAGCTGTTTCCCAGACTCCAAAATACCTGGTCACACAGATGG...,TRB,F,T,T,F,TRBV3-1*01,TRBD1*01,TRBJ2-1*01,...,NaN,B134137,B134703,B18,Endothelial,NaN,TSP8_Prostate_NA_SS2_B134137_B134703_Endotheli...,2317092.0,2209,TSP8_Prostate_NA_SS2_B134137_B134703_Endotheli...
10444,TSP8_Prostate_NA_SS2_B134140_B134704_Epithelia...,TCTCAGACTATTCATCAATGGCCAGCGACCCTGGTGCAGCCTGTGG...,TRB,F,T,T,F,TRBV30*01,NaN,TRBJ1-1*01,...,NaN,B134140,B134704,D7,Epithelial,NaN,TSP8_Prostate_NA_SS2_B134140_B134704_Epithelia...,655334.0,1788,TSP8_Prostate_NA_SS2_B134140_B134704_Epithelia...


In [200]:
meta = pd.read_csv('../data/TSP1_TSP15_metadata.csv')


SS2 = meta[meta.method == 'smartseq2']

SS2[SS2.cell_ontology_class.str.contains('b cell|plasma cell')].groupby('donor').tissue.value_counts()

donor  tissue         
TSP1   Bladder             95
       Pancreas            89
       Blood               60
       Lung                13
TSP10  Fat                 32
       Skin                10
TSP11  Bone_Marrow        256
TSP13  Bone_Marrow         96
TSP2   Lymph_Node         468
       Spleen             350
       Blood              121
       Kidney              67
       Bone_Marrow         65
       Thymus              45
       Bladder             16
       Lung                16
       Skin                11
       Vasculature         10
       Small_Intestine      1
TSP3   Eye                 14
TSP4   Mammary             27
       Uterus              10
TSP5   Eye                 16
TSP6   Trachea             26
       Liver                6
TSP7   Lymph_Node         478
       Spleen             433
       Salivary_Gland     119
       Blood               44
TSP9   Pancreas             1
Name: tissue, dtype: int64

In [183]:
data = pd.read_csv('../data/BCRChangeo.csv')

In [185]:
data.Tissue.value_counts()

vertebralbody    541
LymphNode        426
Spleen           283
Blood            107
Kidney            52
Thymus            36
Bladder           14
Vasculature       11
Lung               8
LungNeuron         7
Muscle             3
LI                 2
SI                 2
Skin               1
Name: Tissue, dtype: int64

In [194]:
for i in data[data.Tissue == 'LymphNode'].SEQUENCE_ID:
    print(i)

bracer_TSP2_LymphNode_supradiaphragmatic_SS2_B113714_B104858_ImmuneLinPos_G20_TRINITY_DN0_c0_g1_i2_H bracer_TSP2_LymphNode_supradiaphragmatic_SS2_B113714_B104858_ImmuneLinPos_G20
bracer_TSP2_LymphNode_supradiaphragmatic_SS2_B113714_B104858_ImmuneLinPos_H2_TRINITY_DN1_c0_g1_i1_H bracer_TSP2_LymphNode_supradiaphragmatic_SS2_B113714_B104858_ImmuneLinPos_H2
bracer_TSP2_LymphNode_Inguinal_SS2_B114663_B133051_ImmuneLinPos_P16_TRINITY_DN0_c0_g1_i1_H bracer_TSP2_LymphNode_Inguinal_SS2_B114663_B133051_ImmuneLinPos_P16
bracer_TSP2_LymphNode_supradiaphragmatic_SS2_B113714_B104858_ImmuneLinPos_L22_TRINITY_DN0_c0_g1_i5_H bracer_TSP2_LymphNode_supradiaphragmatic_SS2_B113714_B104858_ImmuneLinPos_L22
bracer_TSP2_LymphNode_supradiaphragmatic_SS2_B113714_B104858_Endothelial_C14_TRINITY_DN11_c10_g1_i1_H bracer_TSP2_LymphNode_supradiaphragmatic_SS2_B113714_B104858_Endothelial_C14
bracer_TSP2_LymphNode_supradiaphragmatic_SS2_B113714_B104858_ImmuneLinPos_H1_TRINITY_DN0_c0_g1_i3_H bracer_TSP2_LymphNode_supra

In [19]:
merged[merged.locus.str.contains("IG")].tissue.value_counts()

Bone_Marrow    780
Spleen         373
Eye             82
Kidney          60
Trachea         36
Liver           33
Skin            17
Fat             13
Lung             8
Muscle           5
Prostate         3
Bladder          2
Name: tissue, dtype: int64